In [1]:
import pennylane as qml
from pennylane.tape import QuantumTape
from pennylane.operation import Operation
from pennylane.measurements import MeasurementProcess
from pennylane.transforms import create_expand_fn

In [2]:
# les portes de base de notre expansion 
# je triche ici en rajoutant RZ, RX, RY 
# mais c'est surement ok vu que ce qui nous intéresse au final c'est les CNOT
base_gates = ["T", "Adjoint(T)", 
              "S", "Adjoint(S)", 
              "SX", "Adjoint(SX)", 
              "PauliX", "PauliY", "PauliZ", 
              "Hadamard", "CNOT", 
              "RZ", "RX", "RY"]

def decompose(tape : QuantumTape) -> QuantumTape:
    def stop_at(op : Operation):
        return op.name in base_gates

    # pennylane create_expand_fn does the job for us 
    custom_expand_fn = create_expand_fn(depth=9, stop_at=stop_at)
    tape = custom_expand_fn(tape)
    return tape

In [3]:
# définir un device
dev = qml.device("default.qubit", wires = 9)


In [4]:
# le noeud à expandre
def U_A():
    qml.CNOT(wires = [0, 4])
    qml.CNOT(wires = [1, 4])
    qml.CNOT(wires = [2, 5])
    qml.CNOT(wires = [3, 5])
    qml.CNOT(wires = [0, 6])
    qml.CNOT(wires = [2, 6])
    qml.CNOT(wires = [1, 7])
    qml.CNOT(wires = [3, 7])

def U_f():
    U_A()
    qml.MultiControlledX(wires = [4, 5, 6, 7, 8], control_values = [True] * 4)
    U_A()

def W():
    qml.Hadamard(wires = 0)
    qml.Hadamard(wires = 1)
    qml.Hadamard(wires = 2)
    qml.Hadamard(wires = 3)
    qml.MultiControlledX(wires = [0, 1, 2, 3, 8], control_values = [True] * 4)
    qml.Hadamard(wires = 0)
    qml.Hadamard(wires = 1)
    qml.Hadamard(wires = 2)
    qml.Hadamard(wires = 3)

def circuit():
    with QuantumTape() as tape:
        qml.Hadamard(wires = 0)
        qml.Hadamard(wires = 1)
        qml.Hadamard(wires = 2)
        qml.Hadamard(wires = 3)
        qml.PauliX(wires = 8)
        qml.Hadamard(wires = 8)
        U_f()
        W()
        U_f()
        W()
    return tape

In [5]:
# une fonction utilitaire qui crée un noeud à partir d'un tape
# ya surement un meilleur moyen de faire ça, mais c'est ce que j'ai
@qml.qnode(dev)
def arbitrary_circuit(tape : QuantumTape, measurement = qml.counts):
    """
    create a quantum function out of a tape and a default measurement to use (overrides the measurements in the tape)
    """
    for op in tape.operations:
        if len(op.parameters) > 0:
            qml.apply(op)
        else:
            qml.apply(op)
    
    def get_wires(mp : MeasurementProcess):
        return [w for w in mp.wires] if mp is not None and mp.wires is not None and len(mp.wires) > 0 else tape.wires

    # retourner une liste de mesures si on a plusieurs mesures, sinon retourner une seule mesure
    return [measurement(wires=get_wires(meas)) for meas in tape.measurements] if len(tape.measurements) > 1 \
        else measurement(wires=get_wires(tape.measurements[0] if len(tape.measurements) > 0 else None))

In [6]:
# exécuter le circuit, récupérer le tape, décomposer le tape
tape = circuit()
tape = decompose(tape)

In [7]:
# créer un circuit à partir du tape décomposé et récupérer les specs
specs = qml.specs(arbitrary_circuit)(tape)
resources = specs["resources"]
# voilà le gate count
print("\n".join([f"{k} : {v}" for k,v in resources.gate_types.items()]))

Hadamard : 101
PauliX : 1
CNOT : 336
RZ : 112
RY : 56
Adjoint(T) : 120
T : 160
